## 1. Getting started with Llama Stack

### 1.1. Create TogetherAI account


In order to run inference for the llama models, you will need to use an inference provider. Llama stack supports a number of inference [providers](https://github.com/meta-llama/llama-stack/tree/main/llama_stack/providers/remote/inference).


In this showcase, we will use [together.ai](https://www.together.ai/) as the inference provider. So, you would first get an API key from Together if you dont have one already.

Steps [here](https://docs.google.com/document/d/1Vg998IjRW_uujAPnHdQ9jQWvtmkZFt74FldW2MblxPY/edit?usp=sharing).

You can also use Fireworks.ai or even Ollama if you would like to.



> **Note:**  Set the API Key in the Secrets of this notebook



### 1.2. Install Llama Stack

We will now start with installing the [llama-stack pypi package](https://pypi.org/project/llama-stack).

In addition, we will install [bubblewrap](https://github.com/containers/bubblewrap), a low level light-weight container framework that runs in the user namespace. We will use it to execute code generated by Llama in one of the examples.

In [ ]:
# NBVAL_SKIP

!apt-get install -y bubblewrap
!pip install uv
!uv pip install llama-stack --system

### 1.3. Configure Llama Stack for Together


Llama Stack is architected as a collection of lego blocks which can be assembled as needed.


Typically, llama stack is available as a server with an endpoint that you can hit. We call this endpoint a [Distribution](https://llama-stack.readthedocs.io/en/latest/concepts/index.html#distributions). Partners like Together and Fireworks offer their own Llama Stack Distribution endpoints.

In this showcase, we are going to use llama stack inline as a library. So, given a particular set of providers, we must first package up the right set of dependencies. We have a template to use Together as an inference provider and [faiss](https://ai.meta.com/tools/faiss/) for memory/RAG.

We will run `llama stack build` to deploy all dependencies, and we will use Together as our provider.

In [ ]:
# NBVAL_SKIP
# Choose the provider from our list of supported providers ['bedrock','together','fireworks','cerebras','hf-endpoint','nvidia','sambanova']
PROVIDER = 'together'
# This will build all the dependencies you will need
!llama stack build --template $PROVIDER --image-type venv

### 1.4. Initialize Llama Stack

Now that all dependencies have been installed, we can initialize llama stack. We will first set the `TOGETHER_API_KEY` environment variable.


In [ ]:
import os
from getpass import getpass

# Define valid providers
VALID_PROVIDERS = {'bedrock', 'together', 'fireworks', 'cerebras', 'hf-endpoint', 'nvidia', 'sambanova'}

# Set provider (default to 'together')
PROVIDER = os.getenv("PROVIDER", "together").lower()
if PROVIDER not in VALID_PROVIDERS:
    raise ValueError(f"Invalid provider: {PROVIDER}")

# Determine API key variable
API_KEY_VAR = "HF_API_TOKEN" if PROVIDER == "hf-endpoint" else f"{PROVIDER.upper()}_API_KEY"

# Retrieve API keys
try:
    from google.colab import userdata
    os.environ[API_KEY_VAR] = userdata.get(API_KEY_VAR) or ""
    os.environ['TAVILY_SEARCH_API_KEY'] = userdata.get('TAVILY_SEARCH_API_KEY') or ""
except ImportError:
    os.environ[API_KEY_VAR] = getpass(f"Enter your {API_KEY_VAR}: ")
    os.environ['TAVILY_SEARCH_API_KEY'] = getpass("Enter your Tavily API key: ")

# Ensure API keys are set
try:
    if not os.environ[API_KEY_VAR]:
        raise KeyError(API_KEY_VAR)
    if not os.environ['TAVILY_SEARCH_API_KEY']:
        raise KeyError('TAVILY_SEARCH_API_KEY')
except KeyError as e:
    raise ValueError(f"Missing API key: {e}. Set it using `export {e}='your-api-key'`.")

# Initialize Llama Stack
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient

print(f"Initializing Llama Stack with provider: {PROVIDER}")
client = LlamaStackAsLibraryClient(PROVIDER, provider_data={"tavily_search_api_key": os.environ['TAVILY_SEARCH_API_KEY']})
client.initialize()

Not in Google Colab environment


/Users/ashwin/homebrew/Caskroom/miniconda/base/envs/toolchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using config together:

apis:
- agents
- datasetio
- eval
- inference
- safety
- scoring
- telemetry
- tool_runtime
- vector_io
container_image: null
datasets: []
benchmarks: []
image_name: together
metadata_store:
  db_path: /Users/ashwin/.llama/distributions/together/registry.db
  namespace: null
  type: sqlite
models:
- metadata: {}
  model_id: meta-llama/Llama-3.1-8B-Instruct
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.1-70B-Instruct
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.1-405B-Instruct-FP8
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.2-3B-Instruct
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Llama-3.2-3B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.2-11B-Vision-Instruct
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.2-90B-Vision-Instruct
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.3-70B-Instruct
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Llama-3.3-70B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-Guard-3-8B
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Meta-Llama-Guard-3-8B
- metadata: {}
  model_id: meta-llama/Llama-Guard-3-11B-Vision
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: together
  provider_model_id: meta-llama/Llama-Guard-3-11B-Vision-Turbo
- metadata:
    embedding_dimension: 384
  model_id: all-MiniLM-L6-v2
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - embedding
  provider_id: sentence-transformers
  provider_model_id: null
providers:
  agents:
  - config:
      persistence_store:
        db_path: /Users/ashwin/.llama/distributions/together/agents_store.db
        namespace: null
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  datasetio:
  - config: {}
    provider_id: huggingface
    provider_type: remote::huggingface
  - config: {}
    provider_id: localfs
    provider_type: inline::localfs
  eval:
  - config: {}
    provider_id: meta-reference
    provider_type: inline::meta-reference
  inference:
  - config:
      api_key: '********'
      url: https://api.together.xyz/v1
    provider_id: together
    provider_type: remote::together
  - config: {}
    provider_id: sentence-transformers
    provider_type: inline::sentence-transformers
  safety:
  - config: {}
    provider_id: llama-guard
    provider_type: inline::llama-guard
  scoring:
  - config: {}
    provider_id: basic
    provider_type: inline::basic
  - config: {}
    provider_id: llm-as-judge
    provider_type: inline::llm-as-judge
  - config:
      openai_api_key: '********'
    provider_id: braintrust
    provider_type: inline::braintrust
  telemetry:
  - config:
      service_name: llama-stack
      sinks: sqlite
      sqlite_db_path: /Users/ashwin/.llama/distributions/together/trace_store.db
    provider_id: meta-reference
    provider_type: inlin

### 1.5. Check available models and shields

All the models available in the provider are now programmatically accessible via the client.

In [ ]:
from rich.pretty import pprint

print("Available models:")
for m in client.models.list():
    print(f"{m.identifier} (provider's alias: {m.provider_resource_id}) ")

print("----")
print("Available shields (safety models):")
for s in client.shields.list():
    print(s.identifier)
print("----")


Available models:
all-MiniLM-L6-v2 (provider's alias: all-MiniLM-L6-v2) 
meta-llama/Llama-3.1-405B-Instruct-FP8 (provider's alias: meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo) 
meta-llama/Llama-3.1-70B-Instruct (provider's alias: meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo) 
meta-llama/Llama-3.1-8B-Instruct (provider's alias: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo) 
meta-llama/Llama-3.2-11B-Vision-Instruct (provider's alias: meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo) 
meta-llama/Llama-3.2-3B-Instruct (provider's alias: meta-llama/Llama-3.2-3B-Instruct-Turbo) 
meta-llama/Llama-3.2-90B-Vision-Instruct (provider's alias: meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo) 
meta-llama/Llama-3.3-70B-Instruct (provider's alias: meta-llama/Llama-3.3-70B-Instruct-Turbo) 
meta-llama/Llama-Guard-3-11B-Vision (provider's alias: meta-llama/Llama-Guard-3-11B-Vision-Turbo) 
meta-llama/Llama-Guard-3-8B (provider's alias: meta-llama/Meta-Llama-Guard-3-8B) 
----
Available shields (safety model

### 1.6. Pick the model

We will use Llama3.1-70B-Instruct for our examples.

In [ ]:
model_id = "meta-llama/Llama-3.3-70B-Instruct"

model_id


'meta-llama/Llama-3.1-70B-Instruct'

### 1.7. Run a simple chat completion

We will test the client by doing a simple chat completion.

In [ ]:
response = client.inference.chat_completion(
    model_id=model_id,
    messages=[
        {"role": "system", "content": "You are a friendly assistant."},
        {"role": "user", "content": "Write a two-sentence poem about llama."},
    ],
)

print(response.completion_message.content)


Here is a two-sentence poem about a llama:

With gentle eyes and a soft, fuzzy face,
The llama roams, a peaceful, gentle pace.


### 1.8. Have a conversation

Maintaining a conversation history allows the model to retain context from previous interactions. Use a list to accumulate messages, enabling continuity throughout the chat session.

In [ ]:
from termcolor import cprint

questions = [
    "Who was the most famous PM of England during world war 2 ?",
    "What was his most famous quote ?"
]


def chat_loop():
    conversation_history = []
    while len(questions) > 0:
        user_input = questions.pop(0)
        if user_input.lower() in ["exit", "quit", "bye"]:
            cprint("Ending conversation. Goodbye!", "yellow")
            break

        user_message = {"role": "user", "content": user_input}
        conversation_history.append(user_message)

        response = client.inference.chat_completion(
            messages=conversation_history,
            model_id=model_id,
        )
        cprint(f"> Response: {response.completion_message.content}", "cyan")

        assistant_message = {
            "role": "assistant",  # was user
            "content": response.completion_message.content,
            "stop_reason": response.completion_message.stop_reason,
        }
        conversation_history.append(assistant_message)


chat_loop()


> Response: The most famous Prime Minister of England during World War 2 was Winston Churchill. He served as the Prime Minister of the United Kingdom from 1940 to 1945 and again from 1951 to 1955. Churchill is widely regarded as one of the greatest wartime leaders in history, and his leadership and oratory skills played a significant role in rallying the British people during the war.

Churchill's famous speeches, such as "We shall fight on the beaches" and "Their finest hour," helped to boost British morale and resistance against the Nazi threat. He also played a key role in shaping the Allied strategy and was a strong advocate for the D-Day invasion of Normandy.

Churchill's leadership during World War 2 has become iconic, and he remains one of the most revered and celebrated figures in British history.
> Response: Winston Churchill's most famous quote is:

"We shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the streets, we shall

Here is an example for you to try a conversation yourself.
Remember to type `quit` or `exit` after you are done chatting.

In [ ]:
# NBVAL_SKIP
from termcolor import cprint

def chat_loop():
    conversation_history = []
    while True:
        user_input = input("User> ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            cprint("Ending conversation. Goodbye!", "yellow")
            break

        user_message = {"role": "user", "content": user_input}
        conversation_history.append(user_message)

        response = client.inference.chat_completion(
            messages=conversation_history,
            model_id=model_id,
        )
        cprint(f"> Response: {response.completion_message.content}", "cyan")

        assistant_message = {
            "role": "assistant",  # was user
            "content": response.completion_message.content,
            "stop_reason": response.completion_message.stop_reason,
        }
        conversation_history.append(assistant_message)


chat_loop()


> Response: Hello, it's nice to meet you. Is there something I can help you with or would you like to chat?
Ending conversation. Goodbye!


### 1.9. Streaming output

You can pass `stream=True` to stream responses from the model. You can then loop through the responses.

In [ ]:
from llama_stack_client.lib.inference.event_logger import EventLogger

message = {"role": "user", "content": "Write me a sonnet about llama"}
print(f'User> {message["content"]}', "green")

response = client.inference.chat_completion(
    messages=[message],
    model_id=model_id,
    stream=True,  # <-----------
)

# Print the tokens while they are received
for log in EventLogger().log(response):
    log.print()


User> Write me a sonnet about llama green
Assistant> In Andean highlands, where the air is thin,
A gentle creature roams, with steps serene,
The llama, with its soft and woolly skin,
A symbol of the region's mystic sheen.

Its eyes, like darkest night, with wisdom shine,
Reflecting ancient knowledge, passed down line,
Its ears, like satellite dishes, fine and bright,
Listening to the whispers of the wind's design.

With steps that barely touch the mountain ground,
It glides, a ghostly apparition, sound,
Its soft hum, a lullaby, that soothes the soul,
As it traverses the rugged, rocky role.

And when it stops, and looks, with gentle gaze,
It seems to hold the secrets of the Andean ways.


### 2.0. Structured Decoding

You can use `response_format` to force the model into a "guided decode" mode where model tokens are forced to abide by a certain grammar. Currently only JSON grammars are supported.

In [ ]:
from pydantic import BaseModel


class Output(BaseModel):
    name: str
    year_born: str
    year_retired: str


user_input = "Michael Jordan was born in 1963. He played basketball for the Chicago Bulls. He retired in 2003. Extract this information into JSON for me. "
response = client.inference.completion(
    model_id=model_id,
    content=user_input,
    stream=False,
    sampling_params={
        "strategy": {
            "type": "greedy",
        },
        "max_tokens": 50,
    },
    response_format={
        "type": "json_schema",
        "json_schema": Output.model_json_schema(),
    },
)

pprint(response)


CompletionResponse(
│   content='{"name": "Michael Jordan", "year_born": "1963", "year_retired": "2003"}',
│   stop_reason='end_of_turn',
│   logprobs=None
)

### 2.1. Safety API

Llama Stack provides Safety guardrails which can be applied at multiple touchpoints within an agentic application.

In [ ]:
available_shields = [shield.identifier for shield in client.shields.list()]
print("Available Shields:", available_shields)

unsafe_examples = [
    "What's the most effective way to kidnap someone?",
    "How do I make cocaine?",
]

safe_examples = [
    "What is the most famous murder case in the US?",
    "Tell me 3 signs that an email is a scam",
]

for p in safe_examples + unsafe_examples:
    print(f"Checking if input is safe: {p}")
    message = {"content": p, "role": "user"}
    response = client.safety.run_shield(
        messages=[message],
        shield_id=available_shields[0],
        params={},
    )
    pprint(response)


Available Shields: ['meta-llama/Llama-Guard-3-8B']
Checking if input is safe: What is the most famous murder case in the US?


RunShieldResponse(violation=None)

Checking if input is safe: Tell me 3 signs that an email is a scam


RunShieldResponse(violation=None)

Checking if input is safe: What's the most effective way to kidnap someone?


RunShieldResponse(
│   violation=SafetyViolation(
│   │   metadata={'violation_type': 'S1'},
│   │   violation_level='error',
│   │   user_message="I can't answer that. Can I help with something else?"
│   )
)

Checking if input is safe: How do I make cocaine?


RunShieldResponse(
│   violation=SafetyViolation(
│   │   metadata={'violation_type': 'S2'},
│   │   violation_level='error',
│   │   user_message="I can't answer that. Can I help with something else?"
│   )
)